# Remove Mostly Nulls

In this notebook we do two things:

1. We replace the `loan_status` column with a 0, 1.
2. We remove the columns that have mostly null values.

### Importing Packages

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 200

### Reading-In Data

In [2]:
df_binary = pd.read_csv("data_processed/01_binary.csv", low_memory=False)
df_binary.head(3).T

,0,1,2
id,39460023,38202144,17343169
member_id,NaN,NaN,NaN
loan_amnt,5625.0,25000.0,16000.0
funded_amnt,5625.0,25000.0,16000.0
funded_amnt_inv,5625.0,25000.0,15950.0
term,36 months,36 months,60 months
int_rate,12.39,6.49,19.52
installment,187.89,766.12,419.65
grade,C,A,E
sub_grade,C1,A2,E2


### Create 0,1 Label called `charged_off` and Remove `loan_status`

First, lets check the output of the `pd.get_dummies()` method when applied to the `loan_status` column.

In [3]:
pd.get_dummies(df_binary[["loan_status"]])

,loan_status_Charged Off,loan_status_Fully Paid
0,False,True
1,False,True
2,False,True
3,False,True
4,False,True
...,...,...
594765,False,True
594766,False,True
594767,False,True
594768,False,True


In [4]:
df_binary["charged_off"] = pd.get_dummies(df_binary[["loan_status"]])["loan_status_Charged Off"]

In [5]:
df_binary.drop(columns=["loan_status"], inplace=True)

### Identifying Mostly `NaN` Columns

In [6]:
pd.options.display.max_rows = 200

The following code identifies the `NaN` in each column.

In [7]:
df_binary.isnull().head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,charged_off
0,False,True,False,False,False,False,False,False,False,False,...,True,False,False,True,True,True,True,True,True,False
1,False,True,False,False,False,False,False,False,False,False,...,True,False,False,True,True,True,True,True,True,False
2,False,True,False,False,False,False,False,False,False,False,...,True,False,False,True,True,True,True,True,True,False
3,False,True,False,False,False,False,False,False,False,False,...,True,False,False,True,True,True,True,True,True,False
4,False,True,False,False,False,False,False,False,False,False,...,True,False,False,True,True,True,True,True,True,False


The following code calculates the percentage null in each column.  As we can see there is a big jump in nulls from 13% to 50%.  So I am going to drop all columns that have nulls over 13%

In [8]:
df_percent_nulls = \
pd.DataFrame({
    "percent_nulls":np.round(df_binary.isnull().sum() / len(df_binary) * 100)
}).sort_values("percent_nulls").reset_index()
df_percent_nulls

,index,percent_nulls
0,id,0.0
1,revol_util,0.0
2,total_acc,0.0
3,initial_list_status,0.0
4,out_prncp,0.0
5,out_prncp_inv,0.0
6,total_pymnt,0.0
7,total_pymnt_inv,0.0
8,total_rec_prncp,0.0
9,total_rec_int,0.0


Collecting list of columns that have greater that 13% nulls.

In [9]:
to_drop = list(df_percent_nulls.query("percent_nulls > 13")["index"].values)
to_drop

['mths_since_last_delinq',
 'total_cu_tl',
 'inq_fi',
 'all_util',
 'max_bal_bc',
 'open_rv_24m',
 'total_bal_il',
 'open_il_24m',
 'open_il_12m',
 'open_act_il',
 'open_acc_6m',
 'inq_last_12m',
 'open_rv_12m',
 'mths_since_rcnt_il',
 'il_util',
 'mths_since_recent_revol_delinq',
 'mths_since_last_major_derog',
 'mths_since_recent_bc_dlq',
 'mths_since_last_record',
 'desc',
 'settlement_percentage',
 'settlement_amount',
 'settlement_date',
 'settlement_status',
 'debt_settlement_flag_date',
 'settlement_term',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'sec_app_inq_last_6mths',
 'sec_app_fico_range_high',
 'sec_app_fico_range_low',
 'revol_bal_joint',
 'sec_app_earliest_cr_line',
 'sec_app_mort_acc',
 'sec_app_chargeoff_within_12_mths',
 'sec_app_num_rev_accts',
 'sec_app_open_act_il',
 'sec_app_revol_util',
 'sec_app_open_acc',
 'sec_app_collections_12_mths_ex_med',
 'hardship_payoff_balance_amount',
 'member_id',
 'hardship_last_payment_amount',
 'orig_proje

In [10]:
len(to_drop)

58

### Dropping Mostly Null Columns

In [11]:
df_binary.drop(columns=to_drop, inplace=True)
df_binary.head(1).T

,0
id,39460023
loan_amnt,5625.0
funded_amnt,5625.0
funded_amnt_inv,5625.0
term,36 months
int_rate,12.39
installment,187.89
grade,C
sub_grade,C1
emp_title,Teacher


### Writing to CSV

In [12]:
df_binary.to_csv("data_processed/02_mostly_nulls_removed.csv", index=False)